# read

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/train_df.csv', encoding='cp949')
test  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


# 격자공간고유번호 슬라이싱

In [4]:
for i in range(1, 16):
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')

for i in range(1, 16):
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [5]:
for col in train.columns[6:]:
    print(f'{col} : {train[col].nunique()}')

send_1 : 5
send_2 : 17
send_3 : 64
send_4 : 213
send_5 : 234
send_6 : 234
send_7 : 240
send_8 : 511
send_9 : 1161
send_10 : 2069
send_11 : 2082
send_12 : 3081
send_13 : 3581
send_14 : 3684
send_15 : 3684
rec_1 : 5
rec_2 : 17
rec_3 : 65
rec_4 : 229
rec_5 : 251
rec_6 : 251
rec_7 : 268
rec_8 : 979
rec_9 : 3353
rec_10 : 7739
rec_11 : 7945
rec_12 : 19039
rec_13 : 26039
rec_14 : 27826
rec_15 : 27826


In [6]:
# 유니크 갯수 유사한 컬럼 버림
drop_cols = [
    'send_1', 'send_2', 'send_4', 'send_5', 'send_6', 
    'send_10', 'send_14', 'send_15', 'rec_1', 'rec_2', 
    'rec_4', 'rec_5', 'rec_6', 'rec_10', 'rec_13', 
    'rec_14', 'rec_15', 
]

# 유사지역 운송

In [7]:
for i in range(1, 16):
    train_sim = train[train[f'send_{i}'] == train[f'rec_{i}']].shape[0]
    test_sim  = test[test[f'send_{i}'] == test[f'rec_{i}']].shape[0]
    print(f'send_rec_{i} : train {train_sim}, test {test_sim}')

send_rec_1 : train 177, test 28
send_rec_2 : train 177, test 28
send_rec_3 : train 177, test 28
send_rec_4 : train 132, test 19
send_rec_5 : train 132, test 19
send_rec_6 : train 132, test 19
send_rec_7 : train 129, test 17
send_rec_8 : train 68, test 12
send_rec_9 : train 56, test 6
send_rec_10 : train 49, test 5
send_rec_11 : train 49, test 5
send_rec_12 : train 39, test 2
send_rec_13 : train 39, test 2
send_rec_14 : train 37, test 2
send_rec_15 : train 37, test 2


In [8]:
for i in [3, 4, 7, 8, 9, 10, 12, 14]:
    train[f'sim_INNB_{i}'] = 0
    test[f'sim_INNB_{i}']  = 0
    train.loc[train[f'send_{i}'] == train[f'rec_{i}'], f'sim_INNB_{i}'] = 1
    test.loc[test[f'send_{i}'] == test[f'rec_{i}'], f'sim_INNB_{i}']    = 1

In [9]:
train.loc[:, 'sim_INNB_3':].corr()

,sim_INNB_3,sim_INNB_4,sim_INNB_7,sim_INNB_8,sim_INNB_9,sim_INNB_10,sim_INNB_12,sim_INNB_14
sim_INNB_3,1.000000,0.862966,0.853063,0.618764,0.561414,0.525097,0.468388,0.456206
sim_INNB_4,0.862966,1.000000,0.988525,0.717021,0.650564,0.608480,0.542766,0.528649
sim_INNB_7,0.853063,0.988525,1.000000,0.725345,0.658116,0.615544,0.549067,0.534786
sim_INNB_8,0.618764,0.717021,0.725345,1.000000,0.907315,0.848622,0.756974,0.737286
sim_INNB_9,0.561414,0.650564,0.658116,0.907315,1.000000,0.935312,0.834301,0.812602
sim_INNB_10,0.525097,0.608480,0.615544,0.848622,0.935312,1.000000,0.892003,0.868803
sim_INNB_12,0.468388,0.542766,0.549067,0.756974,0.834301,0.892003,1.000000,0.973991
sim_INNB_14,0.456206,0.528649,0.534786,0.737286,0.812602,0.868803,0.973991,1.000000


# 지역별 운송 빈도

In [10]:
for col in train.columns[6:-4]:
    print(f'##### {col} value_counts')
    print(train[col].value_counts())
    print('-----------------------------')

##### send_1 value_counts
5    25344
4     4984
1      859
2      550
3      263
Name: send_1, dtype: int64
-----------------------------
##### send_2 value_counts
50    25344
41     3950
11      859
30      231
26      187
27      185
43      184
44      175
47      171
45      163
28      132
46      124
42      118
48       99
29       46
31       28
36        4
Name: send_2, dtype: int64
-----------------------------
##### send_3 value_counts
501    25344
414     1887
415      909
412      358
413      352
       ...  
458        4
479        4
422        3
469        2
312        1
Name: send_3, Length: 64, dtype: int64
-----------------------------
##### send_4 value_counts
5011    14003
5013    11341
4148     1203
4159      395
4141      363
        ...  
4672        1
4221        1
4182        1
4129        1
4792        1
Name: send_4, Length: 213, dtype: int64
-----------------------------
##### send_5 value_counts
50110    14003
50130    11341
41480     1203
41590      395
4

In [11]:
train['send_2'].value_counts()

50    25344
41     3950
11      859
30      231
26      187
27      185
43      184
44      175
47      171
45      163
28      132
46      124
42      118
48       99
29       46
31       28
36        4
Name: send_2, dtype: int64

In [12]:
send_freq2_map = {11: 1, 41: 2, 50: 3}
train['send_freq_2'] = train['send_2'].map(send_freq2_map)
test['send_freq_2']  = test['send_2'].map(send_freq2_map)

train['send_freq_2'].fillna(0, inplace=True)
test['send_freq_2'].fillna(0, inplace=True)

In [13]:
train['rec_2'].value_counts()

41    7061
50    6833
11    4984
26    1548
28    1419
48    1397
44    1193
47    1138
45     942
27     911
46     885
42     874
43     770
30     682
29     636
31     544
36     183
Name: rec_2, dtype: int64

In [14]:
rec_freq2_map = {47: 1, 44: 1, 48: 1, 28: 1, 26: 1, 11: 2, 50: 3, 41: 3}
train['rec_freq_2'] = train['rec_2'].map(rec_freq2_map)
test['rec_freq_2']  = test['rec_2'].map(rec_freq2_map)

train['rec_freq_2'].fillna(0, inplace=True)
test['rec_freq_2'].fillna(0, inplace=True)

In [15]:
train[['send_freq_2', 'rec_freq_2']] = train[['send_freq_2', 'rec_freq_2']].astype('int64')
test[['send_freq_2', 'rec_freq_2']]  = test[['send_freq_2', 'rec_freq_2']].astype('int64')

# 분류별 target 평균

In [16]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

# drop

In [17]:
drop_cols = ['index', 'SEND_SPG_INNB', 'REC_SPG_INNB']
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [18]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_1,send_2,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,send_11,send_12,send_13,send_14,send_15,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,rec_11,rec_12,rec_13,rec_14,rec_15,sim_INNB_3,sim_INNB_4,sim_INNB_7,sim_INNB_8,sim_INNB_9,sim_INNB_10,sim_INNB_12,sim_INNB_14,send_freq_2,rec_freq_2
0,6.678694,3.672897,3,1,11,112,1129,11290,112900,1129000,11290000,112900001,1129000014,11290000140,112900001404,1129000014045,11290000140453,112900001404530,5,50,501,5011,50110,501100,5011000,50110002,501100022,5011000220,50110002200,501100022004,5011000220046,50110002200463,501100022004630,0,0,0,0,0,0,0,0,1,3
1,5.407921,4.444134,3,1,11,113,1135,11350,113500,1135000,11350000,113500000,1135000009,11350000090,113500000905,1135000009051,11350000090512,113500000905120,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000178,50110001780,501100017803,5011000178037,50110001780373,501100017803730,0,0,0,0,0,0,0,0,1,3
2,6.678694,10.375479,9,1,11,113,1135,11350,113500,1135000,11350000,113500003,1135000030,11350000300,113500003009,1135000030093,11350000300931,113500003009310,5,50,501,5011,50110,501100,5011000,50110002,501100026,5011000265,50110002650,501100026509,5011000265091,50110002650914,501100026509140,0,0,0,0,0,0,0,0,1,3
3,4.658195,4.297401,10,1,11,115,1154,11545,115450,1154500,11545000,115450000,1154500002,11545000020,115450000201,1154500002014,11545000020142,115450000201420,5,50,501,5011,50110,501100,5011000,50110003,501100031,5011000315,50110003150,501100031508,5011000315087,50110003150874,501100031508740,0,0,0,0,0,0,0,0,1,3
4,4.658195,6.807151,3,1,11,116,1165,11650,116500,1165000,11650000,116500002,1165000021,11650000210,116500002100,1165000021008,11650000210083,116500002100830,5,50,501,5011,50110,501100,5011000,50110001,501100017,5011000177,50110001770,501100017705,5011000177051,50110001770512,501100017705120,0,0,0,0,0,0,0,0,1,3


# corr

In [19]:
train.corr()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_1,send_2,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,send_11,send_12,send_13,send_14,send_15,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,rec_11,rec_12,rec_13,rec_14,rec_15,sim_INNB_3,sim_INNB_4,sim_INNB_7,sim_INNB_8,sim_INNB_9,sim_INNB_10,sim_INNB_12,sim_INNB_14,send_freq_2,rec_freq_2
DL_GD_LCLS_NM,1.000000,0.342224,0.062190,-0.451808,-0.451766,-0.451482,-0.451243,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,-0.451231,0.246313,0.204732,0.203205,0.202915,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,0.202902,-0.004301,-0.001004,-0.000561,-0.004987,-0.012209,-0.011335,-0.009958,-0.009659,-0.370076,0.242896
DL_GD_MCLS_NM,0.342224,1.000000,0.181723,-0.396077,-0.378148,-0.375542,-0.375499,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,-0.375489,0.273254,0.236108,0.234644,0.234346,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.234332,0.059245,0.061940,0.062619,0.041706,0.038635,0.039097,0.033120,0.032901,-0.420906,0.228823
INVC_CONT,0.062190,0.181723,1.000000,-0.065490,-0.059426,-0.058765,-0.058706,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,-0.058702,0.068454,0.063224,0.063267,0.063206,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.063201,0.015316,0.013532,0.013114,0.017200,0.020537,0.018531,0.023366,0.017519,-0.088158,0.036048
send_1,-0.451808,-0.396077,-0.065490,1.000000,0.989566,0.988921,0.988854,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,0.988852,-0.467607,-0.402860,-0.400269,-0.399783,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,-0.399760,0.030325,0.026169,0.025869,0.018764,0.017025,0.015923,0.014204,0.013834,0.810902,-0.402152
send_2,-0.451766,-0.378148,-0.059426,0.989566,1.000000,0.999882,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,0.999866,-0.443547,-0.382131,-0.379674,-0.379211,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,-0.379190,0.028764,0.024823,0.024538,0.017798,0.016149,0.015104,0.013473,0.013122,0.737552,-0.381459
send_3,-0.451482,-0.375542,-0.058765,0.988921,0.999882,1.000000,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,-0.439393,-0.378552,-0.376118,-0.375660,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,-0.375639,0.028495,0.024590,0.024308,0.017632,0.015997,0.014963,0.013347,0.013000,0.733644,-0.377887
send_4,-0.451243,-0.375499,-0.058706,0.988854,0.999866,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.438904,-0.378136,-0.375705,-0.375247,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,-0.375226,0.028429,0.024511,0.024230,0.017583,0.015953,0.014924,0.013316,0.012971,0.733324,-0.377450
send_5,-0.451231,-0.375489,-0.058702,0.988852,0.999866,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.438895,-0.378128,-0.375697,-0.375239,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,0.028429,0.024510,0.024229,0.017583,0.015952,0.014924,0.013316,0.012971,0.733314,-0.377442
send_6,-0.451231,-0.375489,-0.058702,0.988852,0.999866,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.438895,-0.378128,-0.375697,-0.375239,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.375218,-0.37521

# modeling

In [20]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [21]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.5 MB/s 


In [22]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_state=0, random_strength=0)

model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

Learning rate set to 0.070793
0:	learn: 5.7066495	total: 85.6ms	remaining: 1m 25s
1:	learn: 5.6651745	total: 121ms	remaining: 1m
2:	learn: 5.6525854	total: 158ms	remaining: 52.6s
3:	learn: 5.6158801	total: 200ms	remaining: 49.8s
4:	learn: 5.5764275	total: 244ms	remaining: 48.6s
5:	learn: 5.5473427	total: 282ms	remaining: 46.7s
6:	learn: 5.5386264	total: 317ms	remaining: 45s
7:	learn: 5.5314585	total: 358ms	remaining: 44.3s
8:	learn: 5.5049423	total: 396ms	remaining: 43.6s
9:	learn: 5.4985235	total: 445ms	remaining: 44.1s
10:	learn: 5.4746807	total: 479ms	remaining: 43.1s
11:	learn: 5.4695359	total: 501ms	remaining: 41.2s
12:	learn: 5.4635621	total: 541ms	remaining: 41.1s
13:	learn: 5.4422490	total: 577ms	remaining: 40.7s
14:	learn: 5.4381775	total: 618ms	remaining: 40.6s
15:	learn: 5.4336985	total: 660ms	remaining: 40.6s
16:	learn: 5.4138367	total: 693ms	remaining: 40.1s
17:	learn: 5.3931091	total: 727ms	remaining: 39.7s
18:	learn: 5.3895684	total: 762ms	remaining: 39.3s
19:	learn: 5.3

,col,fi
0,DL_GD_LCLS_NM,3.332417
1,DL_GD_MCLS_NM,20.873815
2,send_1,0.000123
3,send_2,0.027451
4,send_3,1.758939
5,send_4,2.902890
6,send_5,9.092206
7,send_6,0.000000
8,send_7,0.635159
9,send_8,5.422063


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

model = CatBoostRegressor(random_state=0, random_strength=0)

gscv = GridSearchCV(
    estimator=model, param_grid={}, 
    scoring='neg_root_mean_squared_error', 
    refit=True, cv=KFold(n_splits=10, shuffle=True, random_state=0)
)

gscv.fit(X, y)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 5.5853187	total: 65.7ms	remaining: 13.1s
5:	learn: 5.5755907	total: 78ms	remaining: 12.9s
6:	learn: 5.5386642	total: 90.2ms	remaining: 12.8s
7:	learn: 5.5096855	total: 106ms	remaining: 13.2s
8:	learn: 5.4785811	total: 125ms	remaining: 13.8s
9:	learn: 5.4505793	total: 138ms	remaining: 13.6s
10:	learn: 5.4287410	total: 150ms	remaining: 13.5s
11:	learn: 5.4215851	total: 164ms	remaining: 13.5s
12:	learn: 5.4162904	total: 178ms	remaining: 13.5s
13:	learn: 5.3967011	total: 190ms	remaining: 13.4s
14:	learn: 5.3918402	total: 202ms	remaining: 13.3s
15:	learn: 5.3869345	total: 216ms	remaining: 13.3s
16:	learn: 5.3692086	total: 228ms	remaining: 13.2s
17:	learn: 5.3650643	total: 240ms	remaining: 13.1s
18:	learn: 5.3613541	total: 252ms	remaining: 13s
19:	learn: 5.3583901	total: 268ms	remaining: 13.1s
20:	learn: 5.3557372	total: 280ms	remaining: 13.1s
21:	learn: 5.3530196	total: 293ms	remaining: 13s
22:	learn: 5.3349998	total: 307ms	remaining: 13s
23:	le

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x7fae6f21cf90>,
             param_grid={}, scoring='neg_root_mean_squared_error')

# sub

In [25]:
sub['INVC_CONT'] = gscv.predict(test)
best1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/logistics_06.csv')
best2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/soryoung_submission_v35.csv')
best3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/soryoung_submission_v36.csv')

In [28]:
pd.concat([sub, best1['INVC_CONT'], best2['INVC_CONT'], best3['INVC_CONT']], axis=1).head(20)

,index,INVC_CONT,INVC_CONT,INVC_CONT,INVC_CONT
0,32000,4.828661,4.528327,4.669381,4.674949
1,32001,4.937001,4.738477,4.837141,4.735952
2,32002,5.371940,5.040101,5.300440,5.289564
3,32003,6.062684,5.868895,6.128668,6.127594
4,32004,5.284318,4.840474,5.411674,5.293542
5,32005,6.328675,6.048482,5.821277,5.961720
6,32006,5.187586,4.982499,5.288457,5.174809
7,32007,5.051021,4.913057,5.203556,5.154796
8,32008,5.663297,5.277072,5.705150,5.805371
9,32009,4.892144,5.274227,4.842111,4.782661


In [29]:
sub.head()

,index,INVC_CONT
0,32000,4.828661
1,32001,4.937001
2,32002,5.371940
3,32003,6.062684
4,32004,5.284318


In [30]:
sub.to_csv('/content/drive/MyDrive/Colab Notebooks/dacon_logistics/sub/logistics_12.csv', index=False)